In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip install extra-keras-datasets

  Created wheel for extra-keras-datasets: filename=extra_keras_datasets-0.1.7-cp36-none-any.whl size=7311 sha256=4eccf9e976a6ab53994b40580fdcf51ac95d86223e01de0fd0669539c872dfdf
  Stored in directory: /root/.cache/pip/wheels/8a/04/95/e8f214025c2e40d5618a443e4ea09df0031050b35460ea007c
Successfully built extra-keras-datasets


In [3]:
work_dir = "drive/My Drive/Training Records/"
impl_type = "CNN"
dataset = "CIFAR100.fine.New"

In [4]:

import numpy as np
import pandas as pd
import pickle
import numpy as np
import random
import time
import csv
import os
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from tqdm import tqdm

#from extra_keras_datasets import emnist

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler, LambdaCallback
from IPython.display import display
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import CSVLogger

In [5]:
list_categories_coarse = ['aquatic mammals', 'fish', 'flowers',	'food containers', 'fruit and vegetables', 'household electrical devices', 
                          'household furniture', 'insects',	'large carnivores',	'large man-made outdoor things', 'large natural outdoor scenes',
                          'large omnivores and herbivores',	'medium-sized mammals',	'non-insect invertebrates',	'people', 'reptiles', 'small mammals',
                          'trees', 'vehicles 1', 'vehicles 2']

list_categories = ['beaver', 'dolphin', 'otter', 'seal', 'whale',
                   'aquarium fish', 'flatfish', 'ray', 'shark', 'trout',
                   'orchids', 'poppies', 'roses', 'sunflowers', 'tulips',
                   'bottles', 'bowls', 'cans', 'cups', 'plates',
                   'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers',
                   'clock', 'computer keyboard', 'lamp', 'telephone', 'television',
                   'bed', 'chair', 'couch', 'table', 'wardrobe',
                   'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach',
                   'bear', 'leopard', 'lion', 'tiger', 'wolf',
                   'bridge', 'castle', 'house', 'road', 'skyscraper',
                   'cloud', 'forest', 'mountain', 'plain', 'sea',
                   'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo',
                   'fox', 'porcupine', 'possum', 'raccoon', 'skunk',
                   'crab', 'lobster', 'snail', 'spider', 'worm',
                   'baby', 'boy', 'girl', 'man', 'woman',
                   'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle',
                   'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel',
                   'maple', 'oak', 'palm', 'pine', 'willow',
                   'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train',
                   'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor']
                   
num_classes = len(list_categories)
print('num_classes =', num_classes)

num_classes = 100


In [6]:
# CIFAR100: Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass 
# to which it belongs):
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode="fine")

print('x_train.shape =',x_train.shape)
print('y_train.shape =',y_train.shape)
print('x_test.shape =',x_test.shape)
print('y_test.shape =',y_test.shape)

x_train.shape = (50000, 32, 32, 3)
y_train.shape = (50000, 1)
x_test.shape = (10000, 32, 32, 3)
y_test.shape = (10000, 1)


In [7]:
list_encoded_classes = np.unique(y_test)
print('len(list_encoded_classes) =', len(list_encoded_classes))
print('list_encoded_classes =', list_encoded_classes)

if list_encoded_classes[0] > 0:
    num_to_sub = list_encoded_classes[0]
    print('num_to_sub =', num_to_sub)

    list_encoded_classes_new_train = []
    for encoded_class in y_train:
        list_encoded_classes_new_train.append(encoded_class-num_to_sub)
    y_train = np.array(list_encoded_classes_new_train)
    
    list_encoded_classes_new_test = []
    for encoded_class in y_test:
        list_encoded_classes_new_test.append(encoded_class-num_to_sub)
    y_test = np.array(list_encoded_classes_new_test)

print('\ny_train.shape =',y_train.shape)
print('y_test.shape =',y_test.shape)

print('\nlen(np.unique(y_test)) =', len(np.unique(y_test)))
print('np.unique(y_test) =', np.unique(y_test))

len(list_encoded_classes) = 100
list_encoded_classes = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]

y_train.shape = (50000, 1)
y_test.shape = (10000, 1)

len(np.unique(y_test)) = 100
np.unique(y_test) = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [8]:
x_train_norm = x_train/255
x_test_norm = x_test/255

In [9]:
final_train_imageset = np.expand_dims(x_train_norm, axis = 4)
final_test_imageset = np.expand_dims(x_test_norm, axis = 4)

y_train2 = y_train # np.expand_dims(y_train, axis = 1)
y_test2 = y_test # np.expand_dims(y_test, axis = 1)

print('final_train_imageset.shape =', final_train_imageset.shape)
print('final_test_imageset.shape =', final_test_imageset.shape)
print('y_train2.shape =', y_train2.shape)
print('y_test2.shape =', y_test2.shape)

final_train_imageset.shape = (50000, 32, 32, 3, 1)
final_test_imageset.shape = (10000, 32, 32, 3, 1)
y_train2.shape = (50000, 1)
y_test2.shape = (10000, 1)


In [10]:
final_train_label = tf.keras.utils.to_categorical(y_train2, num_classes)
final_test_label = tf.keras.utils.to_categorical(y_test2, num_classes)

print('final_train_label.shape =',final_train_label.shape)
print('final_test_label.shape =',final_test_label.shape)

final_train_label.shape = (50000, 100)
final_test_label.shape = (10000, 100)


In [13]:
NUM_NEURONS = 256
NUM_LAYERS = 10
BATCH_SIZE = 32
NUM_EPOCHS = 30
#epochs_completed = 0
LEARNING_RATE = 0.0001
EPSILON = 1e-4
DROPOUT = 0.5

LOSS = 'categorical_crossentropy'
ACTIVATION_FUNCTION = 'relu'
FINAL_ACTIVATION_FUNCTION = 'softmax'
validation_split = 0.1
kernel_size=(1,1)
early_stop_after_epochs = 50

nTry = "2"
# +"_kernel"+str(kernel_size)+"_lr"+str(LEARNING_RATE)+"_batch"+str(BATCH_SIZE)+"_epochs"+str(NUM_EPOCHS)
#checkpointer_name  = "weights_"+dataset+"_"+impl_type+"_nLayers"+str(NUM_LAYERS)+"_nNeurons"+str(NUM_NEURONS)+"_dropout"+str(DROPOUT)+"_batch"+str(BATCH_SIZE)+".hdf5"
checkpointer_name  = "weights."+dataset+"."+impl_type+".batch"+str(BATCH_SIZE)+".nTry."+nTry
log_name = "log."+checkpointer_name[8:]+".log"

print('checkpointer_name =', checkpointer_name)
print('log_name =', log_name)

checkpointer_name = weights.CIFAR100.fine.New.CNN.batch32.nTry.2
log_name = log.CIFAR100.fine.New.CNN.batch32.nTry.2.log


In [ ]:
'''
input_shape = final_train_imageset.shape[1:]

# Input tensor shape
inputs = Input(input_shape)
#x = MaxPooling2D(pool_size=(3,3), strides=(1,1))

x = Conv2D(64, kernel_size=(3,3))(inputs)
x = Dropout(DROPOUT)(x)

x = Flatten()(x)
outputs = Dense(num_classes, activation=FINAL_ACTIVATION_FUNCTION)(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()
#'''

In [14]:
pad = 25

input_shape = final_train_imageset.shape[1:]

# Input tensor shape
inputs = Input(input_shape)
x = ZeroPadding3D(padding=(pad,pad,pad))(inputs)

x = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
#x = Dropout(DROPOUT)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(num_classes, activation=FINAL_ACTIVATION_FUNCTION)(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3, 1)]    0         
_________________________________________________________________
zero_padding3d (ZeroPadding3 (None, 82, 82, 53, 1)     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 82, 80, 51, 32)    320       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 41, 40, 25, 32)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 41, 38, 23, 64)    18496     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 20, 19, 11, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 17, 9, 64)     36928 

In [ ]:
'''
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=final_train_imageset.shape[1:]),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation=FINAL_ACTIVATION_FUNCTION)
])
model.summary()
#'''

In [15]:
optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE, epsilon=EPSILON)
model.compile(
    optimizer=optimizer, #'rmsprop',
    loss=LOSS,
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [16]:
# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath = work_dir+checkpointer_name, 
                               #monitor='val_accuracy',
                               monitor='val_loss',
                               save_weights_only=False,  
                               mode='auto', 
                               verbose = 0, 
                               save_best_only =False
                               )
checkpointer_best = ModelCheckpoint(filepath = work_dir+"best_"+checkpointer_name, 
                                    monitor='val_loss', 
                                    save_weights_only=False,
                                    mode='auto',  
                                    verbose = 1, 
                                    save_best_only = True
                                    )
early_stopping = EarlyStopping(monitor='loss', patience=early_stop_after_epochs)

In [17]:
def checkBestPerformance(epoch, logs):
    log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['val_loss', 'val_accuracy'], engine='python')
    min_val_loss = min(log_data.val_loss.values)
    max_val_acc = max(log_data.val_accuracy.values)

    current_val_acc = logs['val_accuracy']
    current_val_loss = logs['val_loss']

    save_filepath = work_dir+"best_"+checkpointer_name
    if current_val_loss < min_val_loss:
        model.save(filepath = save_filepath)
        print("\nval_loss decreased from", min_val_loss, "to", current_val_loss, ".")

        #list_callbacks = [checkpointer, checkpointer_best, csv_logger, early_stopping]
        #print("\nReturned to previous best checkpoint callback.")

    elif (current_val_loss==min_val_loss) and (current_val_acc>max_val_acc):
        model.save(filepath = save_filepath)
        print("\nval_accuracy increased from", max_val_acc, "to", current_val_acc, ".")

        #list_callbacks = [checkpointer, checkpointer_best, csv_logger, early_stopping]
        #print("\nReturned to previous best checkpoint callback.")

    else:
        pass
        #print("\nPerformance did not improve from existing min_val_loss =", min_val_loss, ", max_val_acc =", max_val_acc, ".")
    #return

In [18]:
epochs_completed = 0
csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)

try:
    log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
    epochs_completed = log_data.shape[0]

    if epochs_completed > 0:
        model = load_model(work_dir+checkpointer_name)
        list_callbacks = [checkpointer, LambdaCallback(on_epoch_end=checkBestPerformance), csv_logger, early_stopping]
        print("epochs_completed =", epochs_completed)
except:
    list_callbacks = [checkpointer, checkpointer_best, csv_logger, early_stopping]

In [ ]:
print("Previously completed epochs =", epochs_completed, "\n")

start_time = time.time()
history = model.fit(final_train_imageset, final_train_label, 
                    shuffle=True, 
                    batch_size = BATCH_SIZE, 
                    epochs = NUM_EPOCHS - epochs_completed, 
                    #steps_per_epoch = 2,
                    validation_split = validation_split, 
                    #validation_data = (final_test_imageset, final_test_label),
                    callbacks=list_callbacks
                    )
elapsed_time = time.time() - start_time 
print("\nTime elapsed: ", elapsed_time)

Previously completed epochs = 0 

Epoch 1/30
1407/1407 [==============================] - 220s 154ms/step - loss: 4.3975 - accuracy: 0.0352 - val_loss: 4.1800 - val_accuracy: 0.0648
INFO:tensorflow:Assets written to: drive/My Drive/Training Records/weights.CIFAR100.fine.New.CNN.batch32.nTry.2/assets

Epoch 00001: val_loss improved from inf to 4.17997, saving model to drive/My Drive/Training Records/best_weights.CIFAR100.fine.New.CNN.batch32.nTry.2
INFO:tensorflow:Assets written to: drive/My Drive/Training Records/best_weights.CIFAR100.fine.New.CNN.batch32.nTry.2/assets
Epoch 2/30
1407/1407 [==============================] - 216s 154ms/step - loss: 4.0065 - accuracy: 0.0843 - val_loss: 3.9158 - val_accuracy: 0.1052
INFO:tensorflow:Assets written to: drive/My Drive/Training Records/weights.CIFAR100.fine.New.CNN.batch32.nTry.2/assets

Epoch 00002: val_loss improved from 4.17997 to 3.91583, saving model to drive/My Drive/Training Records/best_weights.CIFAR100.fine.New.CNN.batch32.nTry.2
IN

In [ ]:
result = model.evaluate(final_test_imageset, final_test_label)
print("Accuracy : {}, Loss: {}".format(result[1], result[0]))

In [ ]:
'''
NUM_NEURONS = 64
NUM_LAYERS = 3
DROPOUT = 0.5
checkpointer_name  = "weights_"+dataset+"_"+impl_type+"_nLayers"+str(NUM_LAYERS)+"_nNeurons"+str(NUM_NEURONS)+"_dropout"+str(DROPOUT)+".hdf5"
log_name = "log_"+checkpointer_name[8:-5]+".log"
log_data = pd.read_csv(work_dir+log_name, sep=',', engine='python') 
print('log_name =', log_name)
#'''

model_loaded = load_model(work_dir+"best_"+checkpointer_name)
print("Loaded "+work_dir+"best_"+checkpointer_name+".")

In [ ]:
'''
Record: 
EMNIST-Digit: AlexNet = 99.21, CapsuleNet = 99.37, VGGNet = 99.62, ResNet = 99.63, ma2019 = 99.75;
---

#'''

result2 = model_loaded.evaluate(final_test_imageset, final_test_label)
print("nLayers: {}, nNeurons: {}, DROPOUT: {}, Test Acc: {}, Test Loss: {}".format(NUM_LAYERS, NUM_NEURONS, DROPOUT, round(result2[1], 4), round(result2[0], 4)))

In [ ]:
#elapsed_time = 13273.57

with open(work_dir+'Records.csv', "a") as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow([checkpointer_name[8:-5], round(result2[1], 4), round(result2[0], 4), elapsed_time])
print("Saved results.")

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model_loaded.predict_generator(final_test_imageset, len(final_test_imageset))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Precision [TP/TP+FP] = The ratio of correctly predicted positive observations to the total predicted positive observations.
# Recall (Sensitivity) [TP/TP+FN] = The ratio of correctly predicted positive observations to the all observations in actual class - 'yes'.
# F1 score [F1 Score = 2*(Recall * Precision) / (Recall + Precision)] = The weighted average of Precision and Recall.
# Support = The number of samples of the true response that lie in that class.
print('Classification Report:')
#target_names = ['Mono', 'Di'] # not ['Di', 'Mono']
print(classification_report(y_test, y_pred)) #, target_names=target_names))

In [ ]:
print('Classification Report')
print(classification_report(y_test, y_pred, target_names=list_categories))

In [ ]:
log_data = pd.read_csv(work_dir+log_name, sep=',', engine='python') 
log_data.shape

In [ ]:
# Getting the model history keys 
#history.history.keys()
log_data.head()

In [ ]:
# plot the training artifacts
title = "Val loss for "+dataset+" "+impl_type+" ("+str(NUM_LAYERS)+" layers, "+str(NUM_NEURONS)+" neurons)"

plt.plot(log_data['loss'])
plt.plot(log_data['val_loss'])
plt.title(title)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')

img_path = work_dir+'Images/vLoss_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)

In [ ]:
title = "Val acc for "+dataset+" "+impl_type+" ("+str(NUM_LAYERS)+" layers, "+str(NUM_NEURONS)+" neurons)"

plt.plot(log_data['accuracy'])
plt.plot(log_data['val_accuracy'])
plt.title(title)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'best')

img_path = work_dir+'Images/vAcc_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)

Image ckeck:

In [ ]:
'''
Record:
EMNIST-Digit: AlexNet = 99.21, VGGNet = 99.62, ResNet = 99.63, CapsuleNet = 99.37, ma2019 = 99.75;
dropout = 0.0, Test Acc = 0., Test Loss = 0.;
dropout = 0.2, Test Acc = 0.9959, Test Loss = 0.0328;
dropout = 0.5, Test Acc = 0.9954, Test Loss = 0.0313;
#'''

'''
DROPOUT = 0.5
checkpointer_name  = "weights_"+dataset+"_"+impl_type+"_nLayers"+str(NUM_LAYERS)+"_nNeurons"+str(NUM_NEURONS)+"_dropout"+str(DROPOUT)+".hdf5"
log_name = "log_"+checkpointer_name[8:-5]+".log"
log_data = pd.read_csv(work_dir+log_name, sep=',', engine='python') 
print('log_name =', log_name)
###

title = "Val loss for "+dataset+" "+impl_type+" ("+str(NUM_LAYERS)+" layers, "+str(NUM_NEURONS)+" neurons)"
plt.plot(log_data['loss'])
plt.plot(log_data['val_loss'])
plt.title(title)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')
plt.show()
#'''